In [20]:
#No me cambien las cosas perros
import pandas as pd
from sodapy import Socrata
import seaborn as sns

#aqui sacamos las tablas de datos de hospitales y desfibriladores 
client = Socrata("analisi.transparenciacatalunya.cat", None)

desfibriladors = client.get("wpud-ukyg")
hospitales = client.get("8gmd-gz7i")

df_desfibriladors = pd.DataFrame.from_records(desfibriladors) 
df_hospitales = pd.DataFrame.from_records(hospitales) 
df_socioeconomic = pd.read_csv("socio_eco.csv")

In [21]:
# Dejar solo columnas necesarias
df = df_socioeconomic[['any', 'municipi', 'concepte', 'valor']]

# Limpiar valores vacíos o '..'
df['valor'] = pd.to_numeric(df['valor'], errors='coerce')

# Pivotar
df_socioeconomic = df.pivot_table(
    index=['any', 'municipi'],
    columns='concepte',
    values='valor',
    aggfunc='first'
).reset_index()

# Opcional: limpiar nombres de columnas
df_socioeconomic.columns.name = None
df_socioeconomic  
#busco el maximo de poblacio jove sense estudis postobligatoris


C:\Users\German\AppData\Local\Temp\ipykernel_27940\3774025934.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['valor'] = pd.to_numeric(df['valor'], errors='coerce')


,any,municipi,estrangers de països de renda baixa o mitjana (%),població amb estudis baixos (%),població jove sense estudis postobligatoris (%),població ocupada (%),renda mitjana per persona (€),treballadors de baixa qualificació (%)
0,2022,Abella de la Conca,7.2,18.5,NaN,64.2,10999.0,NaN
1,2022,Abrera,5.4,12.6,5.9,72.4,15746.0,9.9
2,2022,Agramunt,17.1,17.0,25.6,71.8,13776.0,17.1
3,2022,Aguilar de Segarra,0.3,13.3,NaN,77.3,16814.0,NaN
4,2022,Agullana,8.8,13.9,15.6,70.3,19392.0,NaN
...,...,...,...,...,...,...,...,...
942,2022,Viver i Serrateix,0.6,15.1,NaN,81.9,16415.0,NaN
943,2022,Xerta,4.2,16.4,14.1,59.3,12657.0,11.7
944,2022,Àger,9.9,18.0,15.8,71.4,14602.0,NaN
945,2022,Òdena,4.6,14.3,25.3,71.7,14288.0,11.2


In [ ]:
#vamos a crear un rating socioeconomico por municipio, que tenga en cuenta todos los factores.
#Nomralizamos cada columna y luego hacemos una media ponderada. Quiero hacer  una nueva tabla con estos ratings
df_socioeconomic['']


In [4]:
#PREPARACION DE LAS TABLAS CON DESFIBRILADORES 
df_desfibriladors = df_desfibriladors.dropna(subset=['latitud', 'longitud'])
df_hospitales = df_hospitales.dropna(subset=['latitud', 'longitud'])
# dejo solo  los que contengan la palabra "Salut" en la categoria   
df_hospitales = df_hospitales[df_hospitales['categoria'].str.contains("Salut", na=False)]
# me quedo solo con las columnas longitud y latitud
df_hospitales = df_hospitales[['latitud', 'longitud']]
df_desfibriladors = df_desfibriladors[['latitud', 'longitud']]